<a href="https://colab.research.google.com/github/sabbaninikhitha/Breast_Cancer_detection-/blob/main/QSVC_WBCD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from functools import reduce
warnings.filterwarnings('ignore')

In [13]:
!pip install qiskit pylatexenc qiskit_machine_learning qiskit-machine-learning

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
breast_cancer_dataset =pd.read_csv('/content/drive/MyDrive/data.csv')

In [16]:
breast_cancer_dataset['diagnosis'] = breast_cancer_dataset['diagnosis'].replace(['B', 'M'],[0,1])

In [17]:
breast_cancer_dataset = breast_cancer_dataset.drop(['id', 'Unnamed: 32'], axis=1)

In [18]:
diagnosis_col = breast_cancer_dataset.pop('diagnosis')
breast_cancer_dataset.insert(30, 'diagnosis', diagnosis_col)


In [19]:
breast_cancer_dataset = breast_cancer_dataset.drop(['compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean','radius_se', 'texture_se',
                        'perimeter_se', 'area_se', 'smoothness_se','compactness_se',
                        'concavity_se', 'concave points_se', 'symmetry_se',
                        'fractal_dimension_se', 'radius_worst', 'texture_worst',
                        'perimeter_worst', 'area_worst', 'smoothness_worst',
                        'compactness_worst', 'concavity_worst', 'concave points_worst',
                        'symmetry_worst', 'fractal_dimension_worst'], axis=1)

In [20]:
breast_cancer_dataset.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,1
1,20.57,17.77,132.90,1326.0,0.08474,1
2,19.69,21.25,130.00,1203.0,0.10960,1
3,11.42,20.38,77.58,386.1,0.14250,1
4,20.29,14.34,135.10,1297.0,0.10030,1


In [21]:
target_name = 'diagnosis'
labels = breast_cancer_dataset[target_name]
features = breast_cancer_dataset.drop(target_name, axis=1)

print("Features =\n", features)
print("labels =\n", labels)

Features =
      radius_mean  texture_mean  perimeter_mean  area_mean  smoothness_mean
0          17.99         10.38          122.80     1001.0          0.11840
1          20.57         17.77          132.90     1326.0          0.08474
2          19.69         21.25          130.00     1203.0          0.10960
3          11.42         20.38           77.58      386.1          0.14250
4          20.29         14.34          135.10     1297.0          0.10030
..           ...           ...             ...        ...              ...
564        21.56         22.39          142.00     1479.0          0.11100
565        20.13         28.25          131.20     1261.0          0.09780
566        16.60         28.08          108.30      858.1          0.08455
567        20.60         29.33          140.10     1265.0          0.11780
568         7.76         24.54           47.92      181.0          0.05263

[569 rows x 5 columns]
labels =
 0      1
1      1
2      1
3      1
4      1
      ..


In [22]:
from sklearn.preprocessing import MinMaxScaler

features = MinMaxScaler().fit_transform(features)

In [23]:
from sklearn.model_selection import train_test_split
from qiskit.utils import algorithm_globals

algorithm_globals.random_seed = 123
train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, train_size=0.8, random_state=algorithm_globals.random_seed
)

In [24]:
from sklearn.svm import SVC

svc = SVC()
_ = svc.fit(train_features, train_labels)  # suppress printing the return value

In [25]:
train_score_c4 = svc.score(train_features, train_labels)
test_score_c4 = svc.score(test_features, test_labels)

print(f"Classical SVC on the training dataset: {train_score_c4:.2f}")
print(f"Classical SVC on the test dataset:     {test_score_c4:.2f}")

Classical SVC on the training dataset: 0.94
Classical SVC on the test dataset:     0.94


In [35]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

features = MinMaxScaler(feature_range=(0, np.pi)).fit_transform(features)

train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, train_size=0.75, shuffle=False
)

In [39]:
print(train_features.shape)
print(train_labels.shape)

print(test_features.shape)
print(test_labels.shape)

(426, 5)
(426,)
(143, 5)
(143,)


In [40]:
# number of qubits is equal to the number of features
num_qubits = 5

# number of steps performed during the training procedure
tau = 100

# regularization parameter
C = 1000

In [41]:
from qiskit import BasicAer
from qiskit.circuit.library import ZFeatureMap
from qiskit.utils import algorithm_globals

from qiskit_machine_learning.kernels import FidelityQuantumKernel

algorithm_globals.random_seed = 12345

feature_map = ZFeatureMap(feature_dimension=num_qubits, reps=1)

qkernel = FidelityQuantumKernel(feature_map=feature_map)

In [42]:
from qiskit_machine_learning.algorithms import PegasosQSVC

pegasos_qsvc = PegasosQSVC(quantum_kernel=qkernel, C=C, num_steps=tau)

# training
pegasos_qsvc.fit(train_features, train_labels)
pegasos_score_train = pegasos_qsvc.score(train_features, train_labels)
print(f"QSVC Training Accuracy: {pegasos_score_train}")
# testing
pegasos_score = pegasos_qsvc.score(test_features, test_labels)
print(f"QSVC Testing Accuracy: {pegasos_score}")

QSVC Training Accuracy: 0.8873239436619719
QSVC Testing Accuracy: 0.9300699300699301
